In [53]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BinaryType
import rasterio
from rasterio.windows import Window

In [68]:
def read_tiled_tiff(tiff_path):
  with rasterio.open(tiff_path) as src:
    print(src.meta)
    for band in range(src.meta['count']):
      for _, window in src.block_windows(band):
        data = src.read(window=window)
        binary_data = data.tobytes()

        tile_x, tile_y = window.col_off, window.row_off
        tile_x, tile_y = int(tile_x), int(tile_y)
        yield tile_x, tile_y, binary_data, (band + 1)

In [64]:
for x, y, b, band in read_tiled_tiff('/mnt/data/processed/bio1_tiled.tif'):
  print(x, y,band)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 8143, 'height': 5467, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -32.266806146449994,
       0.0, -0.0083333333, 72.19152712905)}
0 0 1
256 0 1
512 0 1
768 0 1
1024 0 1
1280 0 1
1536 0 1
1792 0 1
2048 0 1
2304 0 1
2560 0 1
2816 0 1
3072 0 1
3328 0 1
3584 0 1
3840 0 1
4096 0 1
4352 0 1
4608 0 1
4864 0 1
5120 0 1
5376 0 1
5632 0 1
5888 0 1
6144 0 1
6400 0 1
6656 0 1
6912 0 1
7168 0 1
7424 0 1
7680 0 1
7936 0 1
0 256 1
256 256 1
512 256 1
768 256 1
1024 256 1
1280 256 1
1536 256 1
1792 256 1
2048 256 1
2304 256 1
2560 256 1
2816 256 1
3072 256 1
3328 256 1
3584 256 1
3840 256 1
4096 256 1
4352 256 1
4608 256 1
4864 256 1
5120 256 1
5376 256 1
5632 256 1
5888 256 1
6144 256 1
6400 256 1
6656 256 1
6912 256 1
7168 256 1
7424 256 1
7680 256 1
7936 256 1
0 512 1
256 512 1
512 512 1
768 512 1
1024 512 1
1280 512 1
1536 512 1
1792 512 1
2048 512 1
2304 512 1
2560 512 1
2816 512 1
3072 512 1
3328 

In [65]:
meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 8143,
 'height': 5467,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0083333333, 0.0, -32.266806146449994,
        0.0, -0.0083333333, 72.19152712905)}

In [69]:
spark = SparkSession.builder \
    .appName("Example") \
    .getOrCreate()

schema = StructType([
    StructField("x", IntegerType(), True),
    StructField("y", IntegerType(), True),
    StructField("filename", StringType(), True),
    StructField("band", IntegerType(), True),
    StructField("binary_data", BinaryType(), True)
])

# Assuming you have a list of TIFF paths
tiff_paths = ['/mnt/data/processed/bio1_tiled.tif']
rows = []

for path in tiff_paths:
    for x, y, data, band in read_tiled_tiff(path):
        
        rows.append(Row(x=x, y=y, filename=os.path.basename(path), band=band, binary_data=data))

df = spark.createDataFrame(rows, schema)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 8143, 'height': 5467, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -32.266806146449994,
       0.0, -0.0083333333, 72.19152712905)}


PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `IntegerType()` can not accept object `bio1_tiled.tif` in type `str`.

In [70]:
df.show()

24/02/10 19:42:21 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 3)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_fun

Py4JJavaError: An error occurred while calling o95.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (10.3.58.11 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/elmo/anaconda3/envs/geolife/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line